In [ ]:
%matplotlib widget

import sys
import os
import copy
from pathlib import Path

SCRIPT_DIR = Path(os.getcwd()).parent
sys.path.append(os.path.dirname(SCRIPT_DIR))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import special
from scipy import optimize

from python.fluorophores import FlMoveBleach
from python.simulators import SimSequencefile

## Helper functions we will need, can be ignored by user. Simply run the cell below and continue.

In [ ]:
def getmsd(x,y,time,maxd,dt):
    x = x.squeeze()
    y = y.squeeze()
    time = time.squeeze()
    
    timeint = np.arange(np.min(time), np.max(time)+dt, dt)
    # timeint = np.linspace(np.min(time), np.max(time), (np.max(time)-np.min(time))/dt)
    xint = np.interp(timeint, time, x)
    yint = np.interp(timeint, time, y)
    msd = np.zeros(maxd)
    for di in range(maxd):
        try:
            msd[di] = np.mean((xint[:-di]-xint[di:])**2+(yint[:-di]-yint[di:])**2)
        except ValueError:
            msd[di] = 0

    return msd, dt

def msdfit(msd,dt,linec):
    msdt = np.arange(0, len(msd)*dt+dt, dt)
    ax = plt.gca()
    ax.plot(msdt[1:]/1e3,msd/1e6,linec)
    fp = np.polyfit(msdt[1:], msd, deg=1)
    fpf = np.poly1d(fp)
    ax.plot(msdt/1e3,fpf(msdt)/1e6,linec+"--")
    Dfit = fp[0]/1000/4  # um^2/s
    off = np.sqrt(fp[1])
    ax.set_xlabel('time (s)')
    ax.set_ylabel('MSD ($\mu$m^2)')

    return Dfit, off, msdt


def maxDiffusion(sim, laserpower, repetitions, Ds=None, dname=None, fig=None):
    if Ds is None:
        Ds = np.hstack([0, 0.1, 0.2, np.arange(0.25, 8.25, 0.25)])
    if dname is None:
        dname = "data"
    if fig is None:
        fig = plt.gcf()

    if isinstance(Ds, list):
        Ds = np.array(Ds)

    # Ds = Ds.squeeze()
    # print(f"Ds.shape: {Ds.shape} len(Ds): {len(Ds)}")

    rmse = np.zeros((len(Ds), repetitions, 3))
    efo = np.zeros((len(Ds), repetitions))
    indlosta = np.zeros((len(Ds), repetitions))

    maxerr = 100

    for d in range(len(Ds)):
        for k in range(repetitions):
            sim.posgalvo = [0, 0, 0]
            sim.posEOD = [0, 0, 0]
            sim.time = 0

            fl = FlMoveBleach()
            fl.photonbudget = np.inf
            fl.brightness = 1000*laserpower

            D = Ds[d]  # um^2/s
            fl.makediffusion(D,0.003)
            sim.fluorophores = fl
            out = sim.runSequence(repetitions=1, resetfluorophores=True)
            err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
            # print("err.shape", err.shape, "min", err.min(), "media", np.median(err), "mean", err.mean(), "max", err.max())
            try:
                # print(err)
                indlost = np.where(err<maxerr)[0][-1]
                filter = (out.loc.itr==np.max(out.loc.itr))
                # print("filter.shape", filter.shape)
                filter[indlost:] = False
                sr=sim.summarize_results(out,display=False,filter=filter)
                rmse[d,k,:] = sr.rmse.squeeze()
                efo[d,k] = np.nanmean(out.loc.efo[filter])
            except IndexError:
                indlost = 0
                rmse[d,k,:] = np.nan
                efo[d,k] = np.nan
            indlosta[d,k] = indlost

    isconverged = indlosta > (sim.sequence['locLimit']-10)  # safe margin
    # print("indlosta")
    # print(indlosta)
    # print("sim.sequence['locLimit']", sim.sequence['locLimit'])
    # print(f"isconverged: {isconverged.shape}")
    # print(isconverged)
    fconverged = np.sum(isconverged,axis=1).squeeze()/repetitions
    # print(f"fconverged: {isconverged.shape}")
    # print(fconverged)

    try:
        startind = np.where(fconverged>0.5)[0][-1]
    except IndexError:
        startind = 0
    startD = Ds[startind]

    def sigmoid(x, D, alpha):
        return 0.5-special.erf(alpha*(x-D))/2  # sigmoidal
    
    def sigmoid_diff(p0, x, y):
        return sigmoid(x, *p0) - y
    
    p0 = [startD, 1]
    # popt, _ = optimize.curve_fit(sigmoid, Ds, fconverged, p0=p0)
    res = optimize.least_squares(sigmoid_diff, p0, args=(Ds, fconverged))
    popt = res.x

    # fig, ax = plt.subplots(1,1)
    # ax.scatter(Ds, fconverged)
    # ax.legend()
    # print(f"Ds: {Ds.shape}, fconverged: {fconverged.shape}")
    # print(f"fit popt: {popt}")
    Dmax = popt[0]

    rmsem = np.nanmean(rmse,axis=1).squeeze()
    efoD0 = np.nanmean(efo[0,:])
    try:
        indconv = np.where(Ds<=Dmax)[0][-1]
        rmseDmax = np.nanmean(rmsem[indconv,:2])

        if np.any(np.array(rmseDmax.shape)!=1):
            rmseDmax = np.nan
    except IndexError:
        rmseDmax = np.nan

    if len(fig.get_axes()) < 1:
        ax = fig.add_subplot(1,3,1)
    else:
        ax = fig.get_axes()[0]

    hp = ax.plot(Ds, fconverged, label=dname)
    # ax.set_xlim(left=0)
    # ax.set_ylim(bottom=0)
    ax.set_ylabel("fraction tracked")
    ax.set_xlabel('Diffusion coefficient $\mu$m$^2$/s')
    ax.set_title(f"Dmax: {Dmax:.2f} $\mu$m$^2$/s")
    ax.plot(Ds,sigmoid(Ds,*popt), color=hp[0].get_color(), linewidth=1, label=f"{dname}fit")
    ax.legend()

    if len(fig.get_axes()) < 2:
        ax = fig.add_subplot(1,3,2)
    else:
        ax = fig.get_axes()[1]
    ax.plot(Ds,np.nanmean(efo,axis=1),color=hp[0].get_color(),label=dname)
    # ax.set_xlim(left=0)
    # ax.set_ylim(bottom=0)
    ax.set_xlabel('Diffusion coefficient $\mu$m$^2$/s')
    ax.set_ylabel("efo kHz")
    ax.legend()

    if len(fig.get_axes()) < 3:
        ax = fig.add_subplot(1,3,3)
    else:
        ax = fig.get_axes()[2]
    
    ax.plot(Ds,np.nanmean(rmsem[:,:2],axis=1),color=hp[0].get_color(),label=dname)
    # ax.set_xlim(left=0)
    # ax.set_ylim(bottom=0)
    ax.set_ylabel("RMSE of converged (nm)")
    ax.set_xlabel('Diffusion coefficient $\mu$m$^2$/s')
    ax.legend()
    
    return Dmax, efoD0, rmseDmax


## Start simulation by loading the simulator class

In [ ]:
sim = SimSequencefile()

In [ ]:
fname = os.path.join(SCRIPT_DIR, "examples", "Tracking_2D.json")
fname2 = os.path.join(SCRIPT_DIR, "settings", "PSFvectorial2D.json") # use a PSF that is defined via a json file
sim.loadsequence(fname, fname2)

In [ ]:
sim.sequence['locLimit'] = 100
sim.makepatterns()
defaultsequence = copy.deepcopy(sim.sequence)
defaultdeadtimes = copy.deepcopy(sim.deadtimes)
defaultestimators = copy.deepcopy(sim.estimators)

In [ ]:
laserpower = 1
updatetime = 0.0025 # ms
repetitions = 20
maxerr = 100

## Plot Example Track

In [ ]:
D = 3.0
sim.posgalvo = [0, 0, 0]
sim.posEOD = [0, 0, 0]
sim.time = 0
fl=FlMoveBleach()
fl.photonbudget = np.inf
fl.brightness = 1000*laserpower
fl.makediffusion(D,updatetime)
sim.fluorophores = fl
out=sim.runSequence(repetitions=1, resetfluorophores=True)
err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
indlost = np.where(err<maxerr)[0][-1]

In [ ]:
fig = plt.figure()
ax = plt.subplot(221)
ax.plot(out.loc.loccounter, out.loc.xnm)
ax.plot(out.loc.loccounter,out.loc.xfl1)
ax.plot(out.loc.loccounter, out.loc.ynm)
ax.plot(out.loc.loccounter, out.loc.yfl1)
ax.plot([indlost,indlost],[np.min(out.loc.ynm),np.max(out.loc.ynm)])
ax.set_xlabel('time (localizations)')
ax.set_ylabel('x position (nm)')
ax.legend(['x estimated position','x fluorophore position','y estimated position','y fluorophore position'])

In [ ]:
ax = plt.subplot(222)
ax.plot(out.loc.xnm[:-1], out.loc.ynm[:-1])
mindiff = np.min(np.diff(out.loc.time,axis=0))
tfl = np.arange(0, out.loc.time[-1]+mindiff, mindiff)
ax.plot(np.interp(tfl,fl.pos[:,0],fl.pos[:,1]),np.interp(tfl,fl.pos[:,0],fl.pos[:,2]))
ax.set_xlabel('x position (nm)')
ax.set_ylabel('y position (nm)')
ax.set_aspect('equal')

## investigate what happens if fluorophore gets lost

In [ ]:
D = 3.0
numtracks = 300
lenwin = 10
averagejump = np.zeros(lenwin)
averageerr = np.zeros(lenwin)
nj = 0
for k in range(numtracks):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time=0
    fl = FlMoveBleach()
    fl.photonbudget = np.inf
    fl.brightness = 1000*laserpower
    fl.makediffusion(D, updatetime)
    sim.fluorophores = fl
    out=sim.runSequence(repetitions=1, resetfluorophores=True)
    
    err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
    jx = np.vstack([np.diff(out.loc.xfl1, axis=0),0])
    jy = np.vstack([np.diff(out.loc.yfl1, axis=0),0])
    jump = np.sqrt(jx**2+jy**2)
    
    indlost = np.where(err<maxerr)[0][-1]
    try:
        averagejump += jump[(indlost-lenwin+3):(indlost+3)].squeeze()
        averageerr += err[(indlost-lenwin+3):(indlost+3)].squeeze()
        nj += 1
    except ValueError:
        pass

In [ ]:
ax = plt.subplot(2,2,3)
nx = np.arange(0,len(averageerr)).T-7
ax.plot(nx,averagejump/nj)
ax.plot(nx,averageerr/nj)
ax.set_xlabel('time (localization)')
ax.set_ylabel('jump, localization error (nm)')
ax.legend(['average jump', 'average error'])

## MSD analysis

In [ ]:
tlast = 0
D = 3.0
# plot example track
msdwin, msdwinr = 10, 1000
msdmf = np.zeros(msdwin)
msdfl = np.zeros(msdwin)
msdflr = np.zeros(msdwinr)
imsd, ir = 0, 0
repetitionsmsd = 100
dtmsd, dtmsdr = None, None
for k in range(repetitionsmsd):
    sim.posgalvo = [0, 0, 0]
    sim.posEOD = [0, 0, 0]
    sim.time = 0
    fl = FlMoveBleach()
    fl.photonbudget = np.inf
    fl.brightness = 1000*laserpower
    fl.makediffusion(D,updatetime)
    sim.fluorophores = fl
    out = sim.runSequence(repetitions=1, resetfluorophores=True)
    err = np.sqrt((out.loc.xnm-out.loc.xfl1)**2+(out.loc.ynm-out.loc.yfl1)**2)
    indlost = np.where(err<maxerr)[0][-1]
    
    msdhfr, dtmsdr = getmsd(fl.pos[:,1],fl.pos[:,2],fl.pos[:,0],msdwinr,fl.pos[1,0]-fl.pos[0,0])
    msdflr += msdhfr
    ir += 1
    
    if indlost > 50:
        time = out.loc.time[:indlost]
        if dtmsd is None:
            dtmsd = np.min(np.diff(time, axis=0))
        xfl = np.interp(time, fl.pos[:,0],fl.pos[:,1])  # should be nearest
        yfl = np.interp(time, fl.pos[:,0],fl.pos[:,2])
        msdhf, dtmsd = getmsd(xfl,yfl,time,msdwin,dtmsd)
        
        msdhm, dtmsd = getmsd(out.loc.xnm[:indlost],out.loc.ynm[:indlost],time,msdwin,dtmsd)
        if not (any(np.isnan(msdhm) | any(np.isnan(msdfl)))):
            msdfl += msdhf
            msdmf += msdhm
            imsd += 1

msdfl=msdfl/imsd
msdmf=msdmf/imsd
msdflr=msdflr/ir

In [ ]:
ax = plt.subplot(224)
Dflr, offflr, _ = msdfit(msdflr,dtmsdr,'m')
Dfl, offfl, _ = msdfit(msdfl,dtmsd,'r')
Dmf, offmf, msdt = msdfit(msdmf,dtmsd,'b')
ax.set_xlim([0, msdt[-1]/1e3])


ax.legend(["fluorophore all",f"D={Dflr:.2f} µm2/s","fluorophore tracked",f"D={Dfl:.2f} µm2/s","Minflux",f"D={Dmf:.2f} µm2/s"])

In [ ]:
fig.tight_layout()

## investigate maximum diffusion coefficient for various conditions

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset
sim.makepatterns()
Dtest = np.hstack([0,0.1, 0.2, np.arange(0.25, 7.25, 0.25)])

laserpowers = np.hstack([0.05, 0.1, np.arange(0.2, 1.0, 0.2), np.arange(1,5.5,0.5)])

Dmaxl = np.zeros_like(laserpowers)
efoD0l = np.zeros_like(laserpowers)
rmseDmaxl = np.zeros_like(laserpowers)

fig = plt.figure(figsize=(10,10))
for k in range(len(laserpowers)):
    Dmaxl[k], efoD0l[k], rmseDmaxl[k] = maxDiffusion(sim, laserpowers[k], repetitions, Dtest, f"{laserpowers[k]:.2f}")
fig.suptitle('laserpowers')
fig.tight_layout()

In [ ]:
figf = plt.figure(figsize=(10,10))

axf1 = figf.add_subplot(3,4,1)
axf1.plot(laserpowers,Dmaxl)
axf1.set_xlabel('laserpower (a.u.)')
axf1.set_ylabel('max Diffusion D µm^2/s')
axf1.set_title('laserpower')
# ax.text(0, 0.35, efoD0l)

axf5 = figf.add_subplot(3,4,5)
axf5.plot(laserpowers,rmseDmaxl)
axf5.set_xlabel('laserpower (a.u.)')
axf5.set_ylabel('rmse at Dmax (nm)')
ax.set_title('laserpower')

axf9 = figf.add_subplot(3,4,9)
axf9.plot(laserpowers,efoD0l)
axf9.set_ylabel('efo (kHz)')
axf9.set_xlabel('laser power (a.u.)')
axf9.set_title('laserpower')

axf10 = figf.add_subplot(3,4,10)
axf10.plot(efoD0l,rmseDmaxl)
axf10.set_xlabel('efo (kHz)')
axf10.set_ylabel('rmse at Dmax (nm)')
axf10.set_title('laserpower')

figf.tight_layout()

## redo with no dead times and faster pattern dwell times

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset
sim.deadtimes.point = 0
sim.deadtimes.estimator = 0
sim.sequence['Itr'][3]['patDwellTime'] = 2e-5  # 20 us
sim.makepatterns()
Dtestf = np.hstack([0, 0.1, 0.2, 0.3, np.arange(0.5, 20.5, 0.5)])

Dmaxlf = np.zeros_like(laserpowers)
efoD0lf = np.zeros_like(laserpowers)
rmseDmaxlf = np.zeros_like(laserpowers)

fig = plt.figure(figsize=(10,10))
for k in range(len(laserpowers)):
    Dmaxlf[k], efoD0lf[k], rmseDmaxlf[k] = maxDiffusion(sim, laserpowers[k], repetitions, Dtestf, f"{laserpowers[k]:.2f}")
fig.suptitle('laserpowers')
fig.tight_layout()

sim.deadtimes=copy.deepcopy(defaultdeadtimes)  # back to default

In [ ]:
axf1.plot(laserpowers,Dmaxlf)
axf1.legend(['standard','no deadtime'])

axf5.plot(laserpowers,rmseDmaxlf)
axf5.legend(['standard','no deadtime'])

axf9.plot(laserpowers,efoD0lf)

axf10.plot(efoD0lf,rmseDmaxlf)

figf.tight_layout()

## L scan pattern size

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset
Ldefault = sim.sequence['Itr'][3]['patGeoFactor']*360
Ls = np.array([40, 75, 100, 150, 200, 250])
laserpowerLnorm=laserpower*Ldefault**2/Ls**2

DmaxL = np.zeros_like(Ls, dtype=float)
efoD0L = np.zeros_like(Ls, dtype=float)
rmseDmaxL = np.zeros_like(Ls, dtype=float)

fig = plt.figure(figsize=(10,5))
for k in range(len(Ls)):
    sim.sequence['Itr'][3]['patGeoFactor'] = Ls[k]/360
    sim.makepatterns()
    DmaxL[k], efoD0L[k], rmseDmaxL[k] = maxDiffusion(sim, laserpowerLnorm[k], repetitions, Dtest, f"{Ls[k]:.2f}")
fig.suptitle('L')
# ax.text(0, 0.35, laserpowerLnorm)
fig.tight_layout()


In [ ]:
axf2 = figf.add_subplot(3,4,2)
axf2.plot(Ls,DmaxL)
axf2.set_xlabel('Scan pattern size L (nm)')
axf2.set_ylabel('max Diffusion D $\mu$m$^2$/s')
axf2.set_title('L')

axf6 = figf.add_subplot(3,4,6)
axf6.plot(Ls,rmseDmaxL)
axf6.set_xlabel('Scan pattern size L (nm)')
axf6.set_ylabel('rmse at Dmax (nm)')
axf6.set_title('L')

## photon limit

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset

photlim = [5, 10, 20, 30, 50, 100, 200]

Dmaxpl = np.zeros_like(photlim, dtype=float)
efoD0pl = np.zeros_like(photlim, dtype=float)
rmseDmaxpl = np.zeros_like(photlim, dtype=float)

fig = plt.figure(figsize=(10,5))
for k in range(len(photlim)):
    sim.sequence['Itr'][3]['phtLimit'] = photlim[k]
    sim.makepatterns()
    Dmaxpl[k], efoD0pl[k], rmseDmaxpl[k] = maxDiffusion(sim, laserpower, repetitions, Dtest, f"{photlim[k]:.2f}")
fig.suptitle('photon limit')
fig.tight_layout()

In [ ]:
axf3 = figf.add_subplot(3,4,3)
axf3.plot(photlim,Dmaxpl)
axf3.set_xlabel('photon limit')
axf3.set_ylabel('max Diffusion D µm^2/s')
axf3.set_title('photon limit')

axf7 = figf.add_subplot(3,4,7)
axf7.plot(photlim,rmseDmaxpl)
axf7.set_xlabel('photon limit')
axf7.set_ylabel('rmse at Dmax (nm)')
axf7.set_title('photon limit')

## dwell time

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset

dwelltimes=[10, 20, 50, 100, 200, 500, 1000] # us

Dmaxdt = np.zeros_like(dwelltimes, dtype=float)
efoD0dt = np.zeros_like(dwelltimes, dtype=float)
rmseDmaxdt = np.zeros_like(dwelltimes, dtype=float)

fig = plt.figure(figsize=(10,5))
for k in range(len(dwelltimes)):
    sim.sequence['Itr'][3]['patDwellTime'] = dwelltimes[k]*1e-6
    sim.makepatterns()
    Dmaxdt[k], efoD0dt[k], rmseDmaxdt[k] = maxDiffusion(sim, laserpower, repetitions, Dtest, f"{dwelltimes[k]:.2f}")
fig.suptitle('pattern dwell time')
fig.tight_layout()

In [ ]:
axf4 = figf.add_subplot(3,4,4)
axf4.plot(dwelltimes,Dmaxdt)
axf4.set_xlabel('pattern dwell time $\mu$s')
axf4.set_ylabel('max Diffusion D $\mu$m$^2$/s')
axf4.set_title('pattern dwell time')

axf8 = figf.add_subplot(3,4,8)
axf8.plot(dwelltimes,rmseDmaxdt)
axf8.set_xlabel('pattern dwell time $\mu$s')
axf8.set_ylabel('rmse at Dmax (nm)')
axf8.set_title('pattern dwell time')

## redo with fast instrument

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset
sim.deadtimes.point = 0
sim.deadtimes.estimator = 0

Dmaxdtf = np.zeros_like(dwelltimes, dtype=float)
efoD0dtf = np.zeros_like(dwelltimes, dtype=float)
rmseDmaxdtf = np.zeros_like(dwelltimes, dtype=float)

fig = plt.figure(figsize=(10,5))
for k in range(len(dwelltimes)):
    sim.sequence['Itr'][3]['patDwellTime'] = dwelltimes[k]*1e-6
    sim.makepatterns()
    Dmaxdtf[k], efoD0dtf[k], rmseDmaxdtf[k] = maxDiffusion(sim, laserpower, repetitions, Dtest, f"{dwelltimes[k]:.2f}")
sim.deadtimes=copy.deepcopy(defaultdeadtimes) # back to default
fig.suptitle('pattern dwell time fast')
fig.tight_layout()

In [ ]:
axf4.plot(dwelltimes,Dmaxdtf)
axf4.legend(['normal','fast'])

axf8.plot(dwelltimes,rmseDmaxdtf)

## damping factor

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence) # reset

dampingfs=[0, 0.2, 0.5, 1, 2, 5, 10]

Dmaxdamp = np.zeros_like(dampingfs, dtype=float)
efoD0damp = np.zeros_like(dampingfs, dtype=float)
rmseDmaxdamp = np.zeros_like(dampingfs, dtype=float)

fig = plt.figure(figsize=(10,5))
for k in range(len(dampingfs)):
    sim.sequence['damping'] = dampingfs[k]
    sim.makepatterns()
    Dmaxdamp[k], efoD0damp[k], rmseDmaxdamp[k] = maxDiffusion(sim, laserpower, repetitions, Dtest, f"{dampingfs[k]:.2f}")
fig.suptitle('damping factor')
fig.tight_layout()

In [ ]:
axf11 = figf.add_subplot(3,4,11)
axf11.plot(dampingfs,Dmaxdamp)
axf11.set_xlabel('pattern dwell time µs')
axf11.set_ylabel('max Diffusion D µm^2/s')
axf11.set_title('damping factor')

axf12 = figf.add_subplot(3,4,12)
axf12.plot(dampingfs,rmseDmaxdamp)
axf12.set_xlabel('pattern dwell time µs')
axf12.set_ylabel('rmse at Dmax (nm)')
axf12.set_title('damping factor')

## compare different scan patterns

In [ ]:
sim.sequence=copy.deepcopy(defaultsequence)
sim.makepatterns()
repetitions = 100
fig = plt.figure(figsize=(10,5))

# reference
Dmaxp6, efop6, _ = maxDiffusion(sim, laserpower,repetitions,None,"hexagon")

# 3 instead of 6 probing positions
sim.sequence['Itr'][3]['Mode']['pattern'] = 'triangle'
sim.makepatterns()
Dmaxp3, efop3, _ = maxDiffusion(sim, laserpower,repetitions,None,"triangle")

# 4 instead of 6 probing positions
sim.sequence['Itr'][3]['Mode']['pattern'] = 'square'
sim.makepatterns()
Dmaxp4, efop4, _ = maxDiffusion(sim, laserpower,repetitions,None,"square")

fig.suptitle("scan patterns")

fig.tight_layout()

In [ ]:
fig = plt.figure(figsize=(10,5))
repetitions=100
# standard
sim.sequence=copy.deepcopy(defaultsequence)
sim.makepatterns()
Dmaxp6, efop6, _ = maxDiffusion(sim, laserpower,repetitions,None,"standard")

# optimized
Lm=150
Ldefault=102
sim.sequence=copy.deepcopy(defaultsequence)
sim.sequence['Itr'][3]['patDwellTime'] = 5e-5  #50 us
sim.sequence['Itr'][3]['Mode']['pattern'] = 'square'
sim.sequence['Itr'][3]['patGeoFactor'] = Lm/360
laserpowerLnorm = laserpower*Ldefault**2/Lm**2
sim.sequence['Itr'][3]['phtLimit'] = 10
sim.makepatterns()
Dmaxopt, efoopt, _ = maxDiffusion(sim, laserpowerLnorm,repetitions,None,"optimized")
fig.suptitle(f"D_s={Dmaxp6:.3f}, D_o={Dmaxopt:.3f}")

fig.tight_layout()

## better estimator

In [ ]:
fig = plt.figure(figsize=(10,5))
# standard
sim.sequence=copy.deepcopy(defaultsequence)
sim.makepatterns()
Dmaxp6b,efop6b, _ = maxDiffusion(sim, laserpower,repetitions,None,"standard")

sim.sequence=copy.deepcopy(defaultsequence)
sim.estimators[3]['function']='est_qLSQiter2D'
sim.estimators[3]['par'][2] = False  # no ccr check
sim.estimators[3]['par'][0] = None
sim.makepatterns()

D0est, efoest, _ = maxDiffusion(sim, laserpower,repetitions,None,"iterLSQ")
sim.estimators = copy.deepcopy(defaultestimators)

fig.tight_layout()